In [1]:
using JuMP, Plots, Gurobi, Revise, Suppressor
includet("draftpolicy.jl")

In [ ]:
#3 and 5 is feasible ~10 minutes 

In [ ]:
N = 3
G = 5
abilities = zeros(N).+1/N
s = Season(N,G,abilities)
pi_t = 10
pi_b = 0
nsims = pi_t - pi_b+1
obj_opt = zeros(nsims)
obj_ntd = zeros(nsims) 
opt_wts = zeros(length(s.matches))
ntd_wts= zeros(length(s.matches))
#a,b, opt_wts, ntd_wts = calc_objective(s,5)
for mult in pi_b:pi_t 
    @suppress_out begin
        i = mult-pi_b+1
        obj_opt[i], obj_ntd[i],opt_wts,ntd_wts = calc_objective(s, mult)
    end
    if mult ==10 
        plot([ opt_wts ntd_wts],xlabel="Match",ylabel="Weight",ylims=[0,0.2],labels=["optimal weights","draft weights"])
        savefig("figures/weights10.pdf")
    end 
end

obj_unif= zeros(nsims) .+ 1/(N-1)
obj_ante= zeros(nsims) .+ 1/(N)
plot(pi_b:pi_t,[ obj_ante obj_unif obj_opt obj_ntd], ylims=[0,1.1], xlabel="B", ylabel="Expected Objective",label=["Ex-Ante Uniform", "Ex-Post Uniform","Optimal","R-NTD"])
savefig("figures/simulated_gain.pdf")



In [44]:
obj_ante

13-element Array{Float64,1}:
 0.3333333333333333
 0.3333333333333333
 0.3333333333333333
 0.3333333333333333
 0.3333333333333333
 0.3333333333333333
 0.3333333333333333
 0.3333333333333333
 0.3333333333333333
 0.3333333333333333
 0.3333333333333333
 0.3333333333333333
 0.3333333333333333

In [18]:
function calc_objective(s, mult)
    println(mult)
    topk = 1 
    N = length(s.teams)
    y = zeros(N) .+ 1/N 
    M = size(s.matches, 1)
    ntd_val = zeros(2^M)
    ntd_wts = zeros(M)
    losers = ones(Int,2^M)
    adjust = true 
    model = Model(with_optimizer(Gurobi.Optimizer))

    #draft rule for all partial histories 
    @variable(model, 0<=x[i=0:M, j=1:2^i, k=1:N]<=1) 
    
    #x = model[:x]
    #FAIR constraint 
    for k in 1:N 
        @constraint(model,x[0,1,k]==1/N)
    end
    
    #PROB constraint 
    for i in 1:M 
        for j in 1:2^i 
            @constraint(model,sum(x[i,j,k] for k in 1:N)==1)
        end
    end

    #DC Constraint under equal ability 
    for i in 0:(M-1) 
        for j in 1:2^i
            for k in 1:N 
                @constraint(model,x[i,j,k] == 1/2*x[i+1,2*j,k]+ 1/2*x[i+1,2*j-1,k])
            end
        end
    end
    
    history = zeros(N) 
    step_forward(model, history, y, adjust, ntd_val, ntd_wts, losers,
                  s, 1, 1, mult, topk)
     
    @objective(model, Max, sum(x[M,j,losers[j]] for j in 1:2^M)/2^M)
    
    optimize!(model)
    println("here")
    opt_wts = zeros(M) 
    for m in 1:M
        wt = 0 
        for j in 1:2^(m-1)
            for n in 1:N 
                wt += abs(value(x[m,2*j-1,n]) - value(x[m-1,j,n]))
                wt += abs(value(x[m,2*j,n]) -  value(x[m-1,j,n]))
            end
        end
        opt_wts[m] = wt/(N*2^m)
    end 
        
    
    return objective_value(model), 
            ntd_val, opt_wts, ntd_wts
    
end

calc_objective (generic function with 1 method)

In [19]:
#adds NTD constraint and calculates draft rule as step forward through 
#tree of all possible season outcomes 
#only for equal abilities, for now 
function step_forward(model, history, y, adjust, ntd_val, ntd_wts, losers, s,m,j,pi,topk)
    M = size(s.matches,1)  
    t1,t2 = s.matches[m,:]
    probw1, probl1 = win_prob_sim(add_win(history,t1), s.teams, 
                    s.matches[(m+1):M,:], s.pwin, topk)
    probw2, probl2 = win_prob_sim(add_win(history,t2), s.teams, 
                    s.matches[(m+1):M,:], s.pwin, topk)
        
    diff1 = pi*(probw1[t1]-probw2[t1])
    diff2 = pi*(probw2[t2]-probw1[t2])
        
    x = model[:x]
    @constraint(model, x[m, 2*j, t1] - x[m, 2*j-1, t1] <= diff1)
    @constraint(model, x[m, 2*j-1, t2] - x[m, 2*j, t2] <= diff2)

    if !adjust || diff1 < (probl2[t1]-probl1[t1]) || diff2 < (probl1[t2]-probl2[t2])
        adjust = false 
        y1 = copy(y)
        y2 = copy(y)
    else 
        y1 = probl1 
        y2 = probl2 
    end
    
    ntd_wts[m] += (mean(abs.(y1.-y)) + mean(abs.(y2.-y)))/2^m 

    if m < M 
        step_forward(model, add_win(history, t1), y1, adjust, 
                     ntd_val, ntd_wts, losers, s, m+1, 2*j-1, pi, topk)
        step_forward(model, add_win(history, t2), y2, adjust, 
                     ntd_val, ntd_wts, losers, s, m+1, 2*j, pi, topk)
    else  
        #which loser for the given history 
        losers[2*j-1] = argmin(add_win(history,t1))
        losers[2*j] = argmin(add_win(history,t2))  
        ntd_val[2*j-1] = y1[argmin(add_win(history,t1))]
        ntd_val[2*j] = y2[argmin(add_win(history,t2))]
    end
end

step_forward (generic function with 1 method)

In [64]:
mean([1,2,3])

2.0